## HOPR Token holders 

In [23]:
import import_ipynb
import importlib
import time
import pandas as pd
import os 
from requests import get, post
import requests
from load_dotenv import load_dotenv 

# import functions   
import token_holders_forum_functions


In [24]:
# Load variables 
load_dotenv('secrets.env')

# dune
api_key_dune = os.environ['API_KEY_DUNE']
header = {"x-dune-api-key" : api_key_dune}

# hopr node
api_host_hopr_node = os.environ['API_HOST_HOPR_NODE']
api_key_hopr_node = os.environ['API_KEY_HOPR_NODE']

if 'API_KEY_DUNE' in os.environ and 'API_HOST_HOPR_NODE' in os.environ and 'API_KEY_HOPR_NODE' in os.environ:
    print(True)
else:
    print(False)

True


### Fetch data from Dune

In [25]:
dune_api_url= token_holders_forum_functions.make_api_url(module="query", action="execute", ID="3244002", base_url="https://api.dune.com/api/v1/")
execution_id = token_holders_forum_functions.execute_query(api_url=dune_api_url, headers=header, engine="medium")

In [26]:
dune_api_status_url= token_holders_forum_functions.make_api_url(module="execution", action="status", ID=execution_id, base_url="https://api.dune.com/api/v1/")

In [27]:
token_holders_forum_functions.wait_for_query_completion(status_url=dune_api_status_url, headers=header, interval=5, timeout=600)

Final state: QUERY_STATE_COMPLETED


{'execution_id': '01HGT4DHMBQ72W8HVAXZPVHSGG',
 'query_id': 3244002,
 'state': 'QUERY_STATE_COMPLETED',
 'submitted_at': '2023-12-04T10:01:42.795923Z',
 'expires_at': '2024-03-03T10:06:32.003939Z',
 'execution_started_at': '2023-12-04T10:01:42.801530351Z',
 'execution_ended_at': '2023-12-04T10:06:32.003938188Z',
 'result_metadata': {'column_names': ['wallet',
   'total_hopr_balance',
   'is_hopr_safe',
   'node_address'],
  'result_set_bytes': 400,
  'total_row_count': 4,
  'datapoint_count': 16,
  'pending_time_millis': 5,
  'execution_time_millis': 289202}}

In [28]:
dune_api_results_url= token_holders_forum_functions.make_api_url(module="execution", action="results", ID=execution_id, base_url="https://api.dune.com/api/v1/")
response_result = token_holders_forum_functions.get_query_results(results_url=dune_api_results_url, headers=header)
data = pd.DataFrame(response_result.json()['result']['rows'])
data.head()

,is_hopr_safe,node_address,total_hopr_balance,wallet
0,Yes,0x31faaa8ce2fe3a22eab7c05167312f853adc0042,158650.000000,0xcd1a97453e3525019cdfa66fb669576fafb2c527
1,Yes,0xfa73af5609d354acd0f6e51f77c04d228c370813,23746.789879,0x8832376a388cfcb58dab0cce249f65b86041e4bc
2,No,None,50822.634100,0x217a6d29abbaceafb36207b4cb25acc148e1fc65
3,No,None,519380.632023,0xd9a00176cf49dfb9ca3ef61805a2850f45cb1d05


### Fetch payment channel information 

In [29]:
topology_data = token_holders_forum_functions.get_unique_nodeAddress_peerId_aggbalance_links(api_host_hopr_node, api_key_hopr_node)

print(len(topology_data))

for key, value in list(topology_data.items())[:5]:
    print(f"{key}: {value}")

398
12D3KooWRbkemN63B4kirDtcEUH6pegkNHV5TxdJRX4LYMgyLysF: {'source_node_address': '0xf7f1f61f8c39b3034e294887e76c6b1ccee2f9e1', 'channels_balance': 1.949999999999996e-10}
12D3KooWAAZ3WnaWMuSQnQSFYnwx7g6W725DYmDY1QASZK1nni5B: {'source_node_address': '0xfa73af5609d354acd0f6e51f77c04d228c370813', 'channels_balance': 460.0}
12D3KooWC7Tub3aRmxUM7XjiozW5paXcySN9gWDDvJwNh98DTdUD: {'source_node_address': '0x07a52c8338edef4b6d3a180c1b75815469ff3a46', 'channels_balance': 1071.0}
12D3KooWNJyWUuu9M9RNU8zTMyqCA8K1GqJwv8tjHsc2oYsJJajD: {'source_node_address': '0x9925b9846e6e289e1c7f8a0554bf6900757491ec', 'channels_balance': 226.2}
12D3KooWKn3bLWMymSMznYt3YRrK8cwmYytEzwM1HoDgqkLcmeSu: {'source_node_address': '0x8de95cdc3291152381f24226422ed9c08beec146', 'channels_balance': 3546.8}


In [30]:
# for value in topology_data.values():
#    if value['source_node_address'] == '0xfa73af5609d354acd0f6e51f77c04d228c370813':
#        print(value)

In [31]:
# convert topology data to dictionary 
values = [v for v in topology_data.values()]
df_topology_data = pd.DataFrame(values, columns=['source_node_address', 'channels_balance'])

### Merge Dune and Network Topology Data

In [32]:
result_df = data.merge(df_topology_data, left_on= 'node_address', right_on='source_node_address', how='left').drop(columns=['source_node_address'])

In [33]:
result_df['total_balance'] = result_df['channels_balance'] + result_df['total_hopr_balance'].fillna(0)
result_df.head()

,is_hopr_safe,node_address,total_hopr_balance,wallet,channels_balance,total_balance
0,Yes,0x31faaa8ce2fe3a22eab7c05167312f853adc0042,158650.000000,0xcd1a97453e3525019cdfa66fb669576fafb2c527,NaN,NaN
1,Yes,0xfa73af5609d354acd0f6e51f77c04d228c370813,23746.789879,0x8832376a388cfcb58dab0cce249f65b86041e4bc,460.0,24206.789879
2,No,None,50822.634100,0x217a6d29abbaceafb36207b4cb25acc148e1fc65,NaN,NaN
3,No,None,519380.632023,0xd9a00176cf49dfb9ca3ef61805a2850f45cb1d05,NaN,NaN


### Save results

In [34]:
column_order = ['wallet', 'total_hopr_balance', 'is_hopr_safe', 'node_address', 'channels_balance', 'total_balance']
result_df = result_df[column_order]

result_df.to_csv('hopr_token_holders.csv', index=False)